In [36]:
import pandas as pd
from geograpy import places 
import re 
import geopy
from geopy.geocoders import ArcGIS 
import numpy as np

import requests
from bs4 import BeautifulSoup



In [5]:
data = pd.read_excel("SanAntonioConcerts.xlsx")

vdf= pd.DataFrame(data)
vdf

,Month,Day,Year,Artist,Venue,Unnamed: 5,Notes,Unnamed: 7
0,June,25,1869.0,General Mason Farewell Party: San Antonio Bras...,NaN,NaN,SAE 06/27/1869 p. 3,NaN
1,July,4,1874.0,San Antonio Brass Band,San Pedro Park,NaN,SAE 07/04/1874 Ad p. 2,NaN
2,October,9,1874.0,Tenth Texas Saengerfest,"San Pedro Springs, Turner Hall",NaN,SAE 10/9/1874 p. 3,NaN
3,October,10,1874.0,Tenth Texas Saengerfest,"Casino Hall, Turner Hall",NaN,SAE 10/9/1874 p. 3,NaN
4,October,11,1874.0,Tenth Texas Saengerfest,"San Pedro Springs, Alamo Plaza, Casino Hall",NaN,SAE 10/9/1874 p. 3,NaN
...,...,...,...,...,...,...,...,...
20099,NaN,NaN,NaN,(03/21/2015) opening of Paper Tiger,NaN,NaN,NaN,NaN
20100,NaN,NaN,NaN,(09/16/2022) San Antonio Philharmonic Orchestr...,NaN,NaN,NaN,NaN
20101,NaN,NaN,NaN,(03/04/2023) Reopening of The Espee (Sunset St...,NaN,NaN,NaN,NaN
20102,NaN,NaN,NaN,^ Willie Nelson's first performance at Gruene ...,NaN,NaN,NaN,NaN


In [6]:
vdf['Year']= vdf['Year'].replace("`", "")

In [7]:
vdf.dtypes

Month          object
Day            object
Year          float64
Artist         object
Venue          object
Unnamed: 5    float64
Notes          object
Unnamed: 7     object
dtype: object

In [13]:
vdf['Year'] = vdf['Year'].fillna(-1).astype(int)

In [14]:
vdf[vdf['Year'] > 1999]

,Month,Day,Year,Artist,Venue,Unnamed: 5,Notes,Unnamed: 7
11781,January,29,2000,"Elida y Avante, Joe Lopez, Los Desperados, Los...",Tejano Stampede,NaN,NaN,NaN
11782,January,30,2000,Jeff Healey Band,Sunset Station,NaN,NaN,NaN
11783,February,5,2000,San Antonio Stock Show and Rodeo: Robert Earl ...,Joe & Harry Freeman Coliseum,NaN,NaN,NaN
11784,February,6,2000,San Antonio Stock Show and Rodeo: Kenny Chesney,Joe & Harry Freeman Coliseum,NaN,NaN,NaN
11785,February,7,2000,San Antonio Stock Show and Rodeo: Brooks & Dunn,Joe & Harry Freeman Coliseum,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
20073,November,19,2023,"Enrique Inglesias, Pitbull, Ricky Martin",AT&T Center,NaN,NaN,NaN
20074,November,25,2023,Small Town Habit,Thirsty Horse Saloon,NaN,NaN,NaN
20075,November,29,2023,Andrea Bocelli,AT&T Center,NaN,NaN,NaN
20076,December,9,2023,Walter Beasley,Buena Vista Theater,NaN,NaN,NaN


In [32]:
# adding address column 
vdf['Address'] = np.nan
vdf['Coordinates'] = np.nan

In [43]:
# Filter the DataFrame based on a specific value range from 'col1' and select 'col2' and 'col3'
filtered_vdf = vdf[(vdf['Year'] >= 2000) & (vdf['Year'] <= 2000)][['Venue', 'Year', 'Address', 'Coordinates']]

filtered_vdf

,Venue,Year,Address,Coordinates
11781,Tejano Stampede,2000,NaN,NaN
11782,Sunset Station,2000,NaN,NaN
11783,Joe & Harry Freeman Coliseum,2000,NaN,NaN
11784,Joe & Harry Freeman Coliseum,2000,NaN,NaN
11785,Joe & Harry Freeman Coliseum,2000,NaN,NaN
...,...,...,...,...
11876,Alamodome,2000,NaN,NaN
11877,Far West Rodeo,2000,NaN,NaN
11878,Far West Rodeo,2000,NaN,NaN
11879,Freeman Coliseum,2000,NaN,NaN
